In [27]:
from vllm.sampling_params import GuidedDecodingParams, SamplingParams
from pydantic import BaseModel,Field
from typing import Literal

In [ ]:

import xgrammar as xgr
import json

class GetWeatherInput(BaseModel):
    """Input schema for get_weather function."""
    location: str 
    unit: Literal["celsius", "fahrenheit"]
    
grammar = xgr.Grammar.from_json_schema(json.dumps(GetWeatherInput.model_json_schema()))

print(f"\n✅ Grammar created: {type(grammar)}")
print(f"   Class: {grammar.__class__.__name__}")
print(f"   Module: {grammar.__class__.__module__}")

print('--------')
print(grammar)



✅ Grammar created: <class 'xgrammar.grammar.Grammar'>
   Class: Grammar
   Module: xgrammar.grammar
--------
basic_escape ::= (([\"\\/bfnrt]) | ("u" [A-Fa-f0-9] [A-Fa-f0-9] [A-Fa-f0-9] [A-Fa-f0-9])) (=(basic_string_sub))
basic_string_sub ::= (("\"") | ([^\0-\x1f\"\\\r\n] basic_string_sub) | ("\\" basic_escape basic_string_sub)) (=([ \n\t]* [,}\]:]))
basic_string ::= (("\"" basic_string_sub)) (=(root_part_0 [ \n\t]* "}"))
root_prop_1 ::= (("\"celsius\"") | ("\"fahrenheit\""))
root_part_0 ::= (([ \n\t]* "," [ \n\t]* "\"unit\"" [ \n\t]* ":" [ \n\t]* root_prop_1)) (=([ \n\t]* "}"))
root ::= (("{" [ \n\t]* "\"location\"" [ \n\t]* ":" [ \n\t]* basic_string root_part_0 [ \n\t]* "}"))



In [32]:
import asyncio
import json

import litellm
from llama_cpp import LlamaGrammar
from pydantic import BaseModel
from torch.cuda import temperature


class Rating(BaseModel):
    rating: int

def get_combined_grammar(response_format, eot_token: str = r"</think>"):
    schema_grammar = LlamaGrammar.from_json_schema(json.dumps(response_format.model_json_schema()))

    anything_but_eot_parts = [
        (f'"{eot_token[:i]}" ' if i > 0 else "") + f"[^{eot_token[i]}]"
        for i in range(len(eot_token))
    ]
    anything_but_eot = " | ".join(anything_but_eot_parts)
    grammar_string = f'thinking ::= ({anything_but_eot})+ "{eot_token}"\n' \
                     f'space ::= " "?\n' \
                     f'root ::= thinking space schema'
    thinking_grammar_str = LlamaGrammar.from_string(grammar_string)._grammar

    schema_grammar_str = schema_grammar._grammar.replace("root", "schema")
    combined_grammar_str = f"{schema_grammar_str}\n{thinking_grammar_str}"
    print(combined_grammar_str)
    return LlamaGrammar.from_string(combined_grammar_str)._grammar


def rate_ai_assistant(example):
    prompt = "This is a description of an AI assistant, rate the quality of the description from 0 (poor) to 5 (clear):\n\n```\n{prompt}\n```"
    messages = [{"role": "user", "content": prompt.format(prompt=example["prompt"])}]

    grammar = get_combined_grammar(Rating)
    return grammar


if __name__ == '__main__':
    rate_ai_assistant({"prompt": "you are an etherium developer"})

integer ::= ("-"? integral-part) space
integral-part ::= [0-9] | [1-9] ([0-9] ([0-9] ([0-9] ([0-9] ([0-9] ([0-9] ([0-9] ([0-9] ([0-9] ([0-9] ([0-9] ([0-9] ([0-9] ([0-9] ([0-9])?)?)?)?)?)?)?)?)?)?)?)?)?)?)?
rating-kv ::= "\"rating\"" space ":" space integer
schema ::= "{" space rating-kv "}" space
space ::= " "?
thinking ::= ([^<] | "<" [^/] | "</" [^t] | "</t" [^h] | "</th" [^i] | "</thi" [^n] | "</thin" [^k] | "</think" [^>])+ "</think>"
space ::= " "?
root ::= thinking space schema


In [ ]:
sample_grammer = """basic_escape ::= (([\"\\/bfnrt]) | ("u" [A-Fa-f0-9] [A-Fa-f0-9] [A-Fa-f0-9] [A-Fa-f0-9])) (=(basic_string_sub))
basic_string_sub ::= (("\"") | ([^\0-\x1f\"\\\r\n] basic_string_sub) | ("\\" basic_escape basic_string_sub)) (=([ \n\t]* [,}\]:]))
basic_string ::= (("\"" basic_string_sub)) (=(root_part_0 [ \n\t]* "}"))
root_prop_1 ::= (("\"celsius\"") | ("\"fahrenheit\""))
root_part_0 ::= (([ \n\t]* "," [ \n\t]* "\"unit\"" [ \n\t]* ":" [ \n\t]* root_prop_1)) (=([ \n\t]* "}"))
root ::= (("{" [ \n\t]* "\"location\"" [ \n\t]* ":" [ \n\t]* basic_string root_part_0 [ \n\t]* "}"))"""

